```
   https://oauth.vk.com/authorize?client_id=54293062&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=video&response_type=token&v=5.199
   ```

In [1]:
input_data = '#access_token=vk1.a.I_AS2dz1dlZ3_kWEhAYr4AwNVtGzA5Xsa6320ciVnhdnroan03n_dr8BiVczbR0ogG-Xdi42D6mwui2q6MYpzEQ_du5pBiiWZt9TcPd8rzIT9X0xJU29FtouFOtmUrg45V3-00nFaa1eTH5zSKWTQ-KDhfunCNBP3DMJt61FyC5Lx3eObD6tU_3VtUor0ONY&expires_in=86400&user_id=431075710'
USER_TOKEN = input_data.split("#access_token=")[1].split("&")[0]


In [2]:
API_VERSION = "5.199"
BASE_URL = "https://api.vk.com/method"



In [3]:
import requests
import pandas as pd
import time
from datetime import datetime


In [4]:
def test_api_connection():
    params = {"access_token": USER_TOKEN, "v": API_VERSION}
    try:
        response = requests.get(f"{BASE_URL}/users.get", params=params)
        result = response.json()
        return "error" not in result
    except:
        return False

test_api_connection()

False

In [5]:
def get_videos_by_search(query, count=200, offset=0):
    params = {
        "q": query,
        "count": min(count, 200),
        "offset": offset,
        "access_token": USER_TOKEN,
        "v": API_VERSION
    }
    try:
        response = requests.get(f"{BASE_URL}/video.search", params=params)
        result = response.json()
        return None if "error" in result else result
    except:
        return None

In [6]:
def parse_video_data(video_item):
    likes = video_item.get('likes', {})
    comments = video_item.get('comments', {})
    reposts = video_item.get('reposts', {})
    
    date_val = video_item.get('date', 0)
    date_published = datetime.fromtimestamp(date_val).strftime('%Y-%m-%d %H:%M:%S') if date_val else None
    
    return {
        'video_id': video_item.get('id'),
        'owner_id': video_item.get('owner_id'),
        'title': video_item.get('title', ''),
        'description': video_item.get('description', ''),
        'duration': video_item.get('duration', 0),
        'views': video_item.get('views', 0),
        'likes': likes.get('count', 0) if isinstance(likes, dict) else likes,
        'comments': comments.get('count', 0) if isinstance(comments, dict) else comments,
        'reposts': reposts.get('count', 0) if isinstance(reposts, dict) else reposts,
        'date': date_val,
        'date_published': date_published,
        'image': video_item.get('image', []),
        'first_frame': video_item.get('first_frame', []),
        'width': video_item.get('width', 0),
        'height': video_item.get('height', 0),
        'player': video_item.get('player', ''),
        'access_key': video_item.get('access_key', ''),
        'platform': 'VK'
    }

In [7]:
def collect_videos_data(search_queries, videos_per_query=200, max_queries=5):
    all_videos = []
    queries = search_queries[:max_queries]

    for query in queries:
        offset = 0
        while offset < videos_per_query:
            result = get_videos_by_search(query, count=200, offset=offset)
            if not result or 'response' not in result:
                break

            items = result['response'].get('items', [])
            if not items:
                break

            for video in items:
                parsed = parse_video_data(video)
                parsed['search_query'] = query
                all_videos.append(parsed)

            offset += len(items)
            if offset >= result['response'].get('count', 0) or offset >= videos_per_query:
                break

            time.sleep(0.5)

        time.sleep(1)

    df = pd.DataFrame(all_videos)
    df = df.drop_duplicates(subset=['video_id'], keep='first')
    return df

In [8]:
# Определяем категории и соответствующие им поисковые запросы
# Для каждой категории используем несколько запросов, чтобы собрать больше видео
# VK API возвращает ограниченное количество результатов (~40-50) на один запрос

category_mapping = {
    'Музыка': ['музыка', 'Моргенштерн', 'Scriptonite', 'МакSим', 'Zivert', 'RASA', 'ЛСП', 'Нурминский', 'Markul', 'Три дня дождя', 'A.V.G.', 'Taylor Swift', 'The Weeknd', 'Drake', 'Bad Bunny', 'Ed Sheeran', 'Ariana Grande', 'Justin Bieber', 'Billie Eilish', 'Coldplay', 'Dua Lipaпесня', 'музыкальный клип', 'Патриотические песни', 'Шаман', 'хит', 'музыка 2024', 'Лучшие песни', 'Топ песни', 'музыка 2023', 'музыка 2022', 'музыка 2021', 'Новогодние хиты', 'Лучшие хиты 90-х', 'Лучшие хиты нулевых', 'детские песни', 'лучшие песни русского рэпа', 'INSTASAMKA новый клип 2025', 'Miyagi & Andy Panda 2025', 'The Weeknd новый трек', 'Markul live 2025', 'Kizaru альбом 2025', 'Скриптонит новый трек 2025', 'песни из тиктока 2025', 'новые русские хиты 2025', 'новые зарубежные хиты 2025', 'Чарт Spotify 2025', 'INSTASAMKA новый клип 2025', 'Miyagi & Andy Panda 2025', 'The Weeknd новый трек', 'Markul live 2025', 'Kizaru альбом 2025', 'Скриптонит новый трек 2025', 'песни из тиктока 2025', 'новые русские хиты 2025', 'новые зарубежные хиты 2025', 'Чарт Spotify 2025', 'топ чарты', 'ICEGERGERT', 'the', 'madk1d', 'темный принц', 'баобаб', 'ivanzolo2004', 'базовый минимум', 'банк', 'oxxxymiron', 'JONY', 'LIDA', 'soft blade', 'Toxi$', 'силуэт', 'Ваня Дмитриенко', 'Аня Пересильд', 'Наследство', 'Мияги и эндшпиль', 'Macan', 'Эндшпиль', 'Баста', 'Моя игра', 'мия бойка', 'матушка земля', 'кухни', 'бонд с кнопкой', 'Григорий лепс', 'Люся Чеботина', 'город 312', 'Niletto', 'Король и шут', 'сигма бой', 'кизару', 'FEDUK', 'Артур пирожков', 'Александр рева', 'BEARWOLF', 'XOLIDAYBOY', 'SQWOZ BAB', 'Михаил Круг', 'наваи', 'хамаи', 'тимати', 'джиган', '52', 'техно', 'электронная музыка', 'музыка для зала', 'метал', 'рок', 'фонк', 'бразильский фонк', 'фонк для зала', 'фонк дрифт', 'лана дель рей', 'лсп', 'лучшая джазовая музыка', 'джаз', 'классическая музыка', 'реп 2025', 'грустная музыка', 'спокойная музыка', 'музыка для сна', 'музыка из сумерек', 'хиты 90х', 'хиты 00х', 'музыка 80х', 'сэм смит', 'INSTASAMKA новый клип 2025', 'Miyagi & Andy Panda 2025', 'The Weeknd новый трек', 'Markul live 2025', 'Kizaru альбом 2025', 'Скриптонит новый трек 2025', 'песни из тиктока 2025', 'новые русские хиты 2025', 'новые зарубежные хиты 2025', 'Чарт Spotify 2025', 'ICEGERGERT', 'the', 'madk1d', 'темный принц', 'баобаб', 'ivanzolo2004', 'базовый минимум', 'банк', 'oxxxymiron', 'JONY', 'LIDA', 'soft blade', 'Toxi$', 'силуэт', 'Ваня Дмитриенко', 'Аня Пересильд', 'Наследство', 'Мияги и эндшпиль', 'Macan', 'Эндшпиль', 'Баста', 'Моя игра', 'мия бойка', 'матушка земля', 'кухни', 'бонд с кнопкой', 'Григорий лепс', 'Люся Чеботина', 'город 312', 'Niletto', 'Король и шут', 'сигма бой', 'кизару', 'FEDUK', 'Артур пирожков', 'Александр рева', 'BEARWOLF', 'XOLIDAYBOY', 'SQWOZ BAB', 'Михаил Круг', 'наваи', 'хамаи', 'тимати', 'джиган', '52', 'техно', 'электронная музыка', 'музыка для зала', 'метал', 'рок', 'фонк', 'бразильский фонк', 'фонк для зала', 'фонк дрифт', 'лана дель рей', 'лсп', 'лучшая джазовая музыка', 'джаз', 'классическая музыка', 'реп 2025', 'грустная музыка', 'спокойная музыка', 'музыка для сна', 'музыка из сумерек', 'хиты 90х', 'хиты 00х', 'музыка 80х', 'сэм смит', 'INSTASAMKA новый клип 2025', 'Miyagi & Andy Panda 2025', 'The Weeknd новый трек', 'Markul live 2025', 'Kizaru альбом 2025', 'Скриптонит новый трек 2025', 'песни из тиктока 2025', 'новые русские хиты 2025', 'новые зарубежные хиты 2025', 'Чарт Spotify 2025', 'ICEGERGERT', 'the', 'madk1d', 'темный принц', 'баобаб', 'ivanzolo2004', 'базовый минимум', 'банк', 'oxxxymiron', 'JONY', 'LIDA', 'soft blade', 'Toxi$', 'силуэт', 'Ваня Дмитриенко', 'Аня Пересильд', 'Наследство', 'Мияги и эндшпиль', 'Macan', 'Эндшпиль', 'Баста', 'Моя игра', 'мия бойка', 'матушка земля', 'кухни', 'бонд с кнопкой', 'Григорий лепс', 'Люся Чеботина', 'город 312', 'Niletto', 'Король и шут', 'сигма бой', 'кизару', 'FEDUK', 'Артур пирожков', 'Александр рева', 'BEARWOLF', 'XOLIDAYBOY', 'SQWOZ BAB', 'Михаил Круг', 'наваи', 'хамаи', 'тимати', 'джиган', '52', 'техно', 'электронная музыка', 'музыка для зала', 'метал', 'рок', 'фонк', 'бразильский фонк', 'фонк для зала', 'фонк дрифт', 'лана дель рей', 'лсп', 'лучшая джазовая музыка', 'джаз', 'классическая музыка', 'реп 2025', 'грустная музыка', 'спокойная музыка', 'музыка для сна', 'музыка из сумерек', 'хиты 90х', 'хиты 00х', 'музыка 80х', 'сэм смит'],
	'Спорт': ['спорт', 'футбол', 'тренировка', 'спортивные новости', 'олимпиада', 'бокс', 'формула 1', 'зарядка', 'апл', 'гранпри', 'чемпионат мира', 'шахматы', 'футбол 2025', 'хоккей', 'НХЛ', 'Интер майями', 'фонбет', 'баскетбол', 'волейбол', 'большой теннис', 'спортивная аналитика', 'ММА', 'WWE', 'Pele', 'UFC', 'видеообзоры матчей', 'чм по футболу', 'лига чемпионов 2025', 'АПЛ обзор матчей', 'топ голов за все время', 'топ голов 2025', 'Манчестер сити 2025', 'Роналду', 'Мбаппе', 'Неймар', 'Зенит СПБ обзор матчей', 'Сборная России по футболу', 'Месси', 'Евро 2024', 'Барселона', 'реал мадрид', 'Баскетбол НБА', 'NBA', 'баскетбол хайлайты', 'баскетбол 2025', 'Стеф Карри 2025', 'Майкл Джордан', 'топ данков 2025', 'баскетбол евролига', 'Леброн Джеймс 2025', 'финал НБА 2025', 'Хоккей КХЛ 2025', 'Хоккей Россия', 'сборная России по хоккею', 'хоккейный матч обзор', 'топ голов КХЛ 2025', 'хоккей обзор', 'Александр Овечкин', 'Овечкин 2025', 'ЮФС 2025', 'UFC хайлайты', 'Макгрегор 2025', 'Хабиб Нурмагомедов', 'бокс нокауты 2025', 'Тайсон бой', 'лучшие моменты ММА 2025', 'тренировка дома', 'тренировка на пресс', 'Йога', 'пилатес', 'тренировка для похудения', 'упражнения для спины', 'утренняя тренировка', 'формула 1 2025', 'Макс ферстаппен', 'F1 обзор', 'lewis hamilton', 'F1 McLaren 2025', 'Чарльз Леклерк', 'автоспорт 2025', 'чм по блицу шахматы 2025', 'Магнус Карлсен', 'Хикару накамура', 'Аниш Гири', 'стрим Шипов', 'Турнир претендентов 2025', 'Матч за шахматную корону', 'Ян непомнящий Ализера Фируджа', 'Даниэль Народицкий стримы', 'футбол 2025', 'хоккей', 'НХЛ', 'Интер майями', 'фонбет', 'баскетбол', 'волейбол', 'большой теннис', 'спортивная аналитика', 'ММА', 'WWE', 'Pele', 'UFC', 'видеообзоры матчей', 'чм по футболу', 'лига чемпионов 2025', 'АПЛ обзор матчей', 'топ голов за все время', 'топ голов 2025', 'Манчестер сити 2025', 'Роналду', 'Мбаппе', 'Неймар', 'Зенит СПБ обзор матчей', 'Сборная России по футболу', 'Месси', 'Евро 2024', 'Барселона', 'реал мадрид', 'Баскетбол НБА', 'NBA', 'баскетбол хайлайты', 'баскетбол 2025', 'Стеф Карри 2025', 'Майкл Джордан', 'топ данков 2025', 'баскетбол евролига', 'Леброн Джеймс 2025', 'финал НБА 2025', 'Хоккей КХЛ 2025', 'Хоккей Россия', 'сборная России по хоккею', 'хоккейный матч обзор', 'топ голов КХЛ 2025', 'хоккей обзор', 'Александр Овечкин', 'Овечкин 2025', 'ЮФС 2025', 'UFC хайлайты', 'Макгрегор 2025', 'Хабиб Нурмагомедов', 'бокс нокауты 2025', 'Тайсон бой', 'лучшие моменты ММА 2025', 'тренировка дома', 'тренировка на пресс', 'Йога', 'пилатес', 'тренировка для похудения', 'упражнения для спины', 'утренняя тренировка', 'формула 1 2025', 'Макс ферстаппен', 'F1 обзор', 'lewis hamilton', 'F1 McLaren 2025', 'Чарльз Леклерк', 'автоспорт 2025', 'чм по блицу шахматы 2025', 'Магнус Карлсен', 'Хикару накамура', 'Аниш Гири', 'стрим Шипов', 'Турнир претендентов 2025', 'Матч за шахматную корону', 'Ян непомнящий Ализера Фируджа', 'Даниэль Народицкий стримы',"Даниэль Народицкий стримы",'теннис 2025','Australian Open 2025','Wimbledon 2025','US Open 2025', 'Ролан Гаррос', 'Новак Джокович','Карлос Алькарас','Ига Швёнтек', 'теннис хайлайты 2025','гольф турнир 2025', 'гольф обучение','серфинг видео', "серф 2025"'скейтбординг 2025', 'BMX трюки 2025','сноуборд фристайл', 'лыжи фрирайд','паркур 2025','экстремальные виды спорта 2025', 'скалолазание', 'альпинизм 2025','прыжки с парашютом', 'дайвинг','фридайвинг','фигурное катание 2025','чемпионат мира по фигурному катанию 2025','гран-при по фигурному катанию', 'финал Гран-при 2025','чемпионат России по фигурному катанию 2025','олимпиада фигурное катание',"олимпийские игры",'Камила Валиева 2025', 'Анна Щербакова 2025', 'Александра Трусова 2025','Елизавета Туктамышева 2025','Аделия Петросян 2025', 'Марк Кондратюк 2025','Даниил Глейхенгауз','Алина Загитова', 'Евгения Медведева','Этери Тутберидзе','Татьяна Тарасова интервью','легкая атлетика 2025', 'марафоны 2025', 'бег мотивация', 'забег 2025','подготовка к марафону 2025', 'как начать бегать', 'бег на 10 км 2025','ночной забег Москва 2025', 'трейлраннинг 2025', 'беговые советы 2025','велоспорт 2025', 'тур де франс 2025', 'велогонщики 2025','триатлон 2025', 'Ironman 2025', 'велотренировки 2025', 'вело спорт','вело гонки 2025', 'вело турнир 2025', 'вело блогеры 2025'],
	'Комедия': ['комедия', 'юмор', 'смех', 'комедийное шоу', 'стендап', 'пародия', 'чбд', 'Павел Воля', 'Азамат Мусагалиев', 'Ваня Усович', 'Большое Шоу', 'Слава Комиссаренко', 'Нурлан Сабуров', 'Сергей Орлов', 'ЧБД новый выпуск 2025', 'Excile шоу', 'Литвин пранки', 'Дима Масленников приколы', 'Хазяева стрим', 'ЧБД новый выпуск 2025', 'Excile шоу', 'Литвин пранки', 'Дима Масленников приколы', 'Хазяева стрим', 'ЧБД новый выпуск', 'шоу кстати', 'шоу прожарка', 'большая прожарка', 'ЧБД Воля', 'ЧБД ургант', 'чбд орлов', 'иксайл', 'Excile новый выпуск', 'Эксайл нарезка', 'стендап 2025', 'Сергей Орлов стендап', 'ургант', 'Саша Малой стендап', 'мемы 2025', 'Нарезки стендап', 'камеди клаб', 'камеди вумен 2025', 'импровизация 2025', ' стендап на тнт', 'скетчи 2025', 'Импровизация тнт', 'Уральские пельмени 2025', 'уральские пельмени лучшее', 'пранки 2025', 'топ пранков', 'юмор года 2025', 'куиджи подкаст', 'Идрак Мирзализаде', 'смешные видео'],
    'Обучение': ['урок', 'туториал', 'обучение с нуля', 'как научиться', 'мастер класс', 'математика', 'алгебра', 'физика', 'информатика', 'Поступашки', 'Анализ данных', 'Python', 'Теория Вероятностей', 'Дискретная математика', 'уроки онлайн 2025подготовка к ЕГЭ 2025', 'подготовка к ОГЭ 2025', 'подготовка к ВПР 2025', 'математика ЕГЭ 2025', 'русский язык ЕГЭ 2025', 'физика ЕГЭ 2025', 'информатика ЕГЭ 2025', 'история ЕГЭ 2025', 'обществознание ЕГЭ 2025', 'подготовка к экзаменам 2025', 'решение заданий ЕГЭ 2025', 'обучающие видео для школьников', 'онлайн школа ЕГЭ 2025', 'школьные лайфхаки 2025', 'Поступашки 2025', 'открытый урок ЕГЭ 2025', 'математика с нуля', 'физика с нуля', 'химия с нуля', 'английский с нуля', 'олимпиадная математика', 'Python обучение 2025', 'курс по питону для начинающих', 'анализ данных обучение', 'SQL обучение 2025', 'Excel обучение', 'Power BI обучение', 'Data Science с нуля', 'машинное обучение для новичков', 'курс по нейросетям 2025', 'Prompt Engineering 2025', 'курс Midjourney 2025', 'курсы по веб-разработке', 'Java обучение 2025', 'HTML CSS обучение 2025', 'Git обучение', 'курс по C++', 'джава с нулякурс по JavaScript 2025', 'курс по SQL', 'курс по React 2025', 'курс по Python pandas 2025', 'английский язык обучение 2025', 'английский для начинающих 2025', 'французский язык обучение', 'испанский язык обучение', 'корейский язык обучение', 'немецкий язык обучение', 'обучение японскому языку', 'разговорный английский 2025', 'учим английский по сериалам', 'английский для путешествий', 'курсы по скорочтению', 'курс по саморазвитию 2025', 'уроки онлайн 2025подготовка к ЕГЭ 2025', 'подготовка к ОГЭ 2025', 'подготовка к ВПР 2025', 'математика ЕГЭ 2025', 'русский язык ЕГЭ 2025', 'физика ЕГЭ 2025', 'информатика ЕГЭ 2025', 'история ЕГЭ 2025', 'обществознание ЕГЭ 2025', 'подготовка к экзаменам 2025', 'решение заданий ЕГЭ 2025', 'обучающие видео для школьников', 'онлайн школа ЕГЭ 2025', 'школьные лайфхаки 2025', 'Поступашки 2025', 'открытый урок ЕГЭ 2025', 'математика с нуля', 'физика с нуля', 'химия с нуля', 'английский с нуля', 'олимпиадная математика', 'Python обучение 2025', 'курс по питону для начинающих', 'анализ данных обучение', 'SQL обучение 2025', 'Excel обучение', 'Power BI обучение', 'Data Science с нуля', 'машинное обучение для новичков', 'курс по нейросетям 2025', 'Prompt Engineering 2025', 'курс Midjourney 2025', 'курсы по веб-разработке', 'Java обучение 2025', 'HTML CSS обучение 2025', 'Git обучение', 'курс по C++', 'джава с нулякурс по JavaScript 2025', 'курс по SQL', 'курс по React 2025', 'курс по Python pandas 2025', 'английский язык обучение 2025', 'английский для начинающих 2025', 'французский язык обучение', 'испанский язык обучение', 'корейский язык обучение', 'немецкий язык обучение', 'обучение японскому языку', 'разговорный английский 2025', 'учим английский по сериалам', 'английский для путешествий', 'курсы по скорочтению', 'курс по саморазвитию 2025'],
	'Новости': ['новости', 'новости сегодня', 'актуальные новости', 'события', 'репортаж', 'первый канал', 'РиаНовости', 'Самые Честные Новости', 'новости сегодня', 'новости России', 'мировые новости', 'новости часа', 'актуальные новости 2025', 'главные новости дня', 'новости недели 2025', 'новости месяца 2025', 'вечерние новости 2025', 'сводка новостей', 'итоги года 2024', 'срочные новости 2025', 'новости онлайн трансляция', 'РБК новости', 'РИА Новости', 'ТАСС новости', 'Первый канал новости', 'НТВ новости', 'BBC новости ', 'CNN', 'Bloomberg ', 'Meduza новости 2025', 'Euronews 2025', 'курс доллара 2025', 'курс евро 2025', 'цены на нефть 2025', 'новости биржи', 'рынок криптовалют', 'новости стартапов', 'новости IT-компаний 2025'],
	'Развлечения': ['развлечения', 'шоу', 'ток шоу', 'развлекательное', 'веселье', 'Лайфхаки', 'challenge', 'a4', '24 часа', 'Кто последний', 'excile', 'Дима Масленников', 'Литвин', 'Хазяева','Не смейся челлендж 2025', 'Try not to laugh 2025','угадай песню челлендж 2025', 'угадай блогера 2025', 'угадай страну 2025', 'угадай мелодию 2025','24 часа челлендж', 'фуд челлендж', 'экстремальные челленджи 2025','угадай по голосу ', 'угадай по эмодзи 2025', 'вопрос на миллион 2025','тест на дружбу 2025', 'игровое шоу 2025','Marmok 2025', 'реакции блогеров 2025', 'видео из TikTok 2025','лайфхаки 2025', 'интересные факты 2025', 'эксперименты 2025','путешествия блогеров 2025', 'влог А4 2025', 'влог Литвин 2025', 'день с блогером','лайфхаки TikTok 2025', 'что если видео 2025','А4 против друзей', 'Литвин расследование 2025','Хазяева новый выпуск', 'Коммент аут 2025', 'вписка подкаст 2025','Импровизация новые выпуски', 'Comedy Club лучшие моменты','Comedy Баттл 2025', 'Шоу Контакты 2025','Пой со мной 2025', 'Подкаст Прожарка', 'Вечерний Ургант','Натальная карта', "Олеся Иванченко", "дорох","Денис Дорохов","квн","квн лучшее", "квн 2025", "квн финал", 'видео из YouTube Shorts 2025','реакция на тикток', 'видео из VK Видео', 'популярные видео Reels 2025','тренды YouTube 2025', 'тренды VK 2025', 'тренд из рилсов 2025','реакции блогеров на тренды', 'реакции ютуберов 2025','социальный эксперимент', 'социальные опросы','эксперименты с едой 2025', 'видео эксперименты 2025','научные опыты 2025', 'пранки в школе 2025', 'пранки на улице 2025','неожиданные реакции людей', 'эксперименты тикток', 'вирусные эксперименты 2025','Дилблин','Сатир','Кокошка','Фрймтеймер','Парадеевич','КликКлак','Куплинов','Марьяна Ро','Настя Ивлеева','Кореш','Данила Горилла','Кто последний выйдет из комнаты?','Выполни безумное испытание','Кто последний уснет','Юрий Дудь','Поперечный','Ваня Усович','Алексей Щербаков','Мистер Бист','IShowSpeed','Пушка шоу 2025', 'Коммент аут 2025', 'Импровизация Комики 2025','Кто я челлендж', 'Кто я шоу', 'Тест на дружбу','угадай блогера 2025', 'угадай по голосу челлендж', 'угадай по фото','Дилблин 2025', 'Дилблин шоу', 'Дилблин реакции', 'Дилблин нарезка','Сатир', 'Сатир пародии', 'Сатир реакции', 'Кокошка шоу', 'Кокошка подкаст', 'Кокошка лучшие моменты', 'Фрймтеймер видео', 'Фрймтеймер нарезки', 'Парадеевич реакции', 'Парадеевич шоу','КликКлак 2025', 'КликКлак новый выпуск','Куплинов летсплей', 'Куплинов реакции', 'Куплинов хоррор','Марьяна Ро влог', 'Марьяна Ро новый выпуск','Настя Ивлеева 2025','Кореш 2025', 'Кореш стрим','Данила Горилла', 'Данила Горилла приколы','Кто последний выйдет из комнаты', 'Кто последний выйдет из круга','Кто последний уснет челлендж 2025', 'Кто последний уснет шоу','Выполни безумное испытание 2025', 'выполни безумное испытание челлендж','Юрий Дудь интервью','Поперечный 2025', 'Поперечный стендап', 'Поперечный подкаст', 'Поперечный концерт','Ваня Усович 2025', 'Ваня Усович стендап','Ваня Усович концерт','Алексей Щербаков 2025','Мистер Бист 2025', 'MrBeast challenge', 'MrBeast new video', 'Мистер Бист челлендж','IShowSpeed 2025', 'IShowSpeed stream', 'IShowSpeed funny moments', 'IShowSpeed'],
	'Игры': ['игры', 'летсплей', 'игровой обзор', 'киберспорт', 'стрим игры', 'Кс', 'CS', 'Counter strike', 'Dota 2', 'major', 'blast premier', 'the international', 'pgl', 'Valorant', 'Champions', 'Minecraft','Counter Strike 2', 'CS2 хайлайты 2025', 'CS2 геймплей 2025', 'Dota 2 The International 2025', 'Dota 2 major 2025','Valorant Champions 2025', 'Valorant турнир 2025', 'Fortnite новый сезон 2025', 'Fortnite челлендж 2025','PUBG 2025', 'PUBG highlights 2025', 'Warzone 2025', 'Warzone нарезка 2025','Apex Legends 2025', 'Apex Legends топ моменты', 'Overwatch 2 gameplay 2025','League of Legends Worlds 2025', 'GTA 6 трейлер', 'GTA 5 RP 2025', 'GTA Online 2025', 'GTA V нарезка 2025','Cyberpunk 2077 Phantom Liberty', 'The Witcher 3 2025', 'Elden Ring 2025','Baldur’s Gate 3 прохождение', 'Hogwarts Legacy 2025', 'Palworld 2025','Stalker 2 2025', 'Atomic Heart DLC 2025', 'Resident Evil 4 Remake 2025','Minecraft 2025', 'Minecraft выживание 2025', 'Minecraft хоррор 2025','Minecraft моды 2025','Roblox 2025', 'Roblox obby 2025', 'Roblox funny moments 2025','Roblox horror 2025', 'Roblox симулятор 2025', 'Roblox тренды 2025','киберспорт 2025', 'esports 2025', 'Blast Premier 2025', 'PGL Major 2025','The International 2025', 'Valorant Masters 2025', 'DreamHack 2025','ESL One 2025', 'Faceit Major 2025', 'IEM Katowice 2025','Fortnite World Cup 2025','CS2 турниры 2025','турнир Dota 2 2025', 'турнир CS2 2025', 'игровые финалы 2025','Kuplinov Play 2025', 'Мормок 2025', 'BlackUFA 2025', 'Evelone 2025', 'Amkal gaming 2025', 'Nikita St4r игры 2025', 'Бустер стрим 2025','BRATISHKIN LIVE 2025', 'SilverName 2025', 'Buster 2025', 'VooDooSh 2025','Игры Twitch 2025','стримы ютуб 2025', 'геймплей стрим 2025', 'летсплей 2025','смоки CS2', 'гайд по Dota 2', 'обучение Valorant', 'гайд по GTA RP','гайд по Minecraft 2025', 'моды Minecraft 2025', 'топ модов 2025','новые игры 2025', 'обзор игр 2025','инди игры 2025', 'лучшие игры 2025', 'игры Steam 2025', 'игры Epic Games 2025','Mobile Legends', 'Clash Royale', 'Brawl Stars 2025','Genshin Impact 2025','Among Us','Standoff 2 ', 'PUBG Mobile 2025', 'Call of Duty Mobile','топ мобильных игр 2025', 'новые мобильные игры 2025',"игры плейстейшн",'Мормок приколы 2025', 'Мормок нарезки', 'Kuplinov хоррор','BlackUFA стрим', 'Evelone игры 2025', 'Бустер стрим 2025','BRATISHKIN стрим 2025', 'SilverName Hearthstone', 'VooDooSh Warzone', 'Nikita St4r игры', 'Amkal gaming','Мистер Бист в играх', 'MrBeast gaming 2025', 'IShowSpeed играет 2025',],
	'Кино': ['кино', 'фильм', 'сериал', 'кино обзор', 'новый фильм', 'кино 2024', 'Крестный отец', 'Побег из Шоушенка', 'Темный рыцарь', 'Криминальное чтиво', 'Клуб "Завтрак"', 'Форрест Гамп', 'Матрица', 'Начало', 'Список Шиндлера', 'Властелин колец: Возвращение короля', 'Титаник', 'Терминатор 2: Судный день', 'Король Лев', 'Назад в будущее', 'Бойцовский клуб', 'Хороший, плохой, злой', 'Пролетая над гнездом кукушки', 'Жизнь прекрасна', 'Леон', 'Игра престолов', 'Во все тяжкие', 'Друзья', 'Офис', 'Шерлок', 'Черное зеркало', 'Очень странные дела', 'Рик и Морти', 'Клан Сопрано', 'Декстер', 'Фарго', 'Настоящий детектив', 'Безумцы', 'Доктор Кто', 'Секретные материалы', 'Карточный домик', 'Мандалорец', 'Корона', 'Кремниевая долина', 'Бриджертоны', 'Гарри Поттер', 'Ужастики', 'Хоррор', 'Сериалы', 'Netflix', '', 'Властелин Колец', 'Звездные войны', 'Disney', 'Кристофер Нолан', 'Квентин Тарантино', 'Фильмы о войне', 'Боевики', 'Фентези', 'Детективы', 'Фантастика', 'блокбастеры'],
	'Кулинария': ['кулинария', 'рецепт', 'готовка', 'кулинарный рецепт', 'обзор на ресторан', 'еда', 'десерт', 'кукинг', 'полезная еда', 'фуд блогер', 'обзор на ресторан', 'рецепт пиццы Маргариты', 'рецепт пасты Карбонара', 'рецепт борща', 'рецепт салата Цезарь', 'рецепт куриного супа', 'рецепт омлета', 'рецепт шашлыка', 'рецепт суши', 'рецепт греческого салата', 'рецепт гуляша', 'рецепт плова', 'рецепт блинов', 'рецепт томатного супа', 'рецепт лазаньи', 'рецепт голубцов', 'рецепт сырников', 'рецепт пельменей', 'рецепт рататуя', 'рецепт чизкейка', 'рецепт шоколадного торта'],
	'Путешествия': ['путешествия', 'отпуск', 'туризм', 'поездка', 'достопримечательности', 'путешествие на поезде', 'путешествие на самолет', 'путешествие на машине', 'пешее путешествие', 'пора валить', 'автостопом', 'Италия', 'Япония', 'Франция', 'Испания', 'Греция', 'Таиланд', 'Мексика', 'Португалия', 'Исландия', 'Новая Зеландия', 'Перу', 'Вьетнам', 'Кения', 'Норвегия', 'Индия', 'Мальдивы', 'Чехия', 'Канада', 'Турция', 'Египет', 'Эйфелева башня', 'Колизей', 'Великая Китайская стена', 'Тадж-Махал', 'Статуя Свободы', 'Мачу-Пикчу', 'Пирамиды Гизы', 'Стоунхендж', 'Сидунейский оперный театр', 'Храм Ангкор-Ват', 'Петра', 'Бурдж-Халифа', 'Акрополь', 'Московский Кремль', 'Биг-Бен', 'Римский форум', 'Статуя Христа-Искупителя', 'Лувр', 'Замок Нойшванштайн', 'Саграда Фамилия','путешествия 2025', 'туризм 2025', 'новые маршруты 2025','топ стран для отдыха 2025', 'куда поехать летом 2025', 'куда поехать зимой 2025','горящие туры 2025', 'лучшие курорты 2025', 'экзотические страны 2025','европейские города 2025', 'отдых у моря 2025', 'лучшие пляжи 2025','путешествия в Россию 2025', 'внутренний туризм 2025', 'тур по Золотому кольцу 2025','поездка на Байкал 2025', 'отдых на Алтае 2025', 'тур по Кавказу 2025','Сочи отдых 2025', 'Крым туризм 2025', 'Калининград путешествие 2025','ОАЭ 2025', 'Катар 2025', 'Саудовская Аравия 2025', 'Турция 2025', 'Грузия 2025','Армения 2025', 'Казахстан 2025', 'Узбекистан 2025', 'Киргизия 2025','Сингапур 2025', 'Малайзия 2025', 'Филиппины 2025', 'Индонезия 2025','Бали 2025', 'Дубай 2025', 'Абу-Даби 2025', 'Сеул 2025', 'Париж 2025','Нью-Йорк 2025', 'Лос-Анджелес 2025', 'Берлин 2025', 'Прага 2025', 'Рим 2025','Барселона 2025', 'Токио 2025', 'Осака 2025', 'Хельсинки 2025', 'Цюрих 2025','Копенгаген 2025', 'Амстердам 2025', 'Будапешт 2025', 'Дубровник 2025','Антон Птушкин путешествия', 'Птушкин 2025', 'Птушкин новые видео','Орёл и Решка 2025', 'Орёл и Решка новый сезон', 'Орёл и Решка Европа 2025','Африка глазами блогера', 'Путешествия с Птушкиным', 'влог путешествие 2025','Литвин отдых', 'A4 путешествия 2025','влог из отпуска 2025', 'влог в другой стране 2025', 'путешествие с друзьями 2025','путешествия по миру 2025', 'блогеры путешественники 2025','тревел подкаст 2025', 'истории путешественников 2025','поход в горы 2025', 'кемпинг 2025', 'трекинг 2025', 'туризм с палаткой 2025','альпинизм 2025', 'сплав по реке 2025', 'рафтинг 2025', 'дайвинг 2025','серфинг 2025', 'сноубординг 2025', 'лыжи 2025', 'поход выходного дня 2025','путешествие автостопом 2025', 'road trip 2025', 'путешествие на машине 2025','жизнь в доме на колесах 2025', 'караван туризм 2025','как собрать чемодан 2025', 'лайфхаки для путешествий 2025','дешевые авиабилеты 2025', 'поиск отелей 2025', 'лучшие авиакомпании 2025','туризм без визы 2025', 'туризм с детьми 2025', 'туризм для пожилых 2025','поездка с питомцами 2025', 'путешествия без денег 2025', 'работа во время путешествий 2025','путешествия для фрилансеров', 'цифровые кочевники 2025', 'travel vlog 2025','уличная еда 2025', 'кухня разных стран 2025', 'национальные блюда 2025','вкусы мира 2025', 'еда в путешествиях 2025', 'где поесть в Париже','ночная жизнь в Токио', 'достопримечательности Европы 2025','самые красивые места мира', 'фотогеничные места 2025', 'лучшие виды 2025','панорамы городов 2025', 'атмосфера старых городов', 'путешествия по музеям 2025','влог о путешествиях 2025', 'travel vlog 2025', 'видео о странах 2025','видео о природе 2025', 'видео о горах 2025', 'видео о морях 2025','видео из Европы 2025', 'видео из Азии 2025', 'видео из Америки 2025','путешествие в Африку 2025', 'видео о пустыне 2025', 'экспедиции 2025','документальное путешествие 2025', 'фильм о путешествиях 2025','видео тревел 2025', 'топ путешествий 2025'],
	'Технологии': ['технологии', 'гаджеты', 'техника', 'новые технологии', 'обзор техники', 'Samsung', 'Apple', 'Xiaomi', 'Huawei', 'OnePlus', 'Google', 'Oppo', 'Vivo', 'Motorola', 'Realme', 'ChatGPT', 'Midjourney', 'Claude', 'Stable Diffusion', 'DALL-E', 'Gemini', 'Copilot', 'Sora', 'Jasper', 'Perplexity', 'Intel', 'AMD', 'NVIDIA', 'ASUS', 'Corsair', 'Wylsacom', 'Drozdoff Tech', 'Эльдар Муртазин', 'iXBT', 'Apple Insider', 'Илон Маск', 'Марк Цукерберг', 'Джефф Безос', 'Дженсен Хуанг', 'Сэм Альтман', 'WhatsApp', 'Telegram', 'Instagram', 'YouTube', 'TikTok', 'Spotify', 'Google Maps', 'Uber', 'Duolingo', 'Notion', 'Google Chrome', 'Microsoft Word', 'Photoshop', 'Zoom', 'Spotify', 'Visual Studio Code', 'Notion', 'Steam', 'Telegram', 'Figma'],
	'Здоровье': ['здоровье', 'фитнес', 'диета', 'здоровый образ жизни', 'медицина', 'спартак как быть здоровым', 'интервальное голодание', 'топ витаминов чтоб быть здоровым', 'как соблюдать зож', 'люди с лучшим здоровьем'],
	 'Автомобили': ['автомобили', 'машины', 'авто обзор', 'автомобиль', 'авто новости', 'Toyota', 'BMW', 'Mercedes', 'Audi', 'Honda', 'Ford', 'Volkswagen', 'Nissan', 'Hyundai', 'Kia', 'Chevrolet', 'Volvo', 'Mazda', 'Lexus', 'Subaru', 'Porsche', 'Ferrari', 'Jeep', 'Tesla', 'Skoda', 'Lada', 'Haval', 'машина москвич', 'обзор на авто буханку'],
	'Животные': ['животные', 'кошки', 'собаки', 'питомцы', 'животные видео', 'звери', 'Кот', 'Собака', 'Слон', 'Тигр', 'Пингвин', 'Кенгуру', 'Лиса', 'Дельфин', 'Орёл', 'Медведь', 'Жираф', 'Зебра', 'Волк', 'Панда', 'Лев', 'Обезьяна', 'Змея', 'Сова', 'Ёж', 'Бегемот'],
	'Танцы': ['танцы', 'танцевальный', 'хореография', 'dance', 'танцевальный батл', 'вы в танцах', 'танцы под фонарем', 'шоу танцы 1 сезон', 'шоу танцы 2 сезон', 'шоу танцы 3 сезон', 'шоу танцы 4 сезон', 'танец до потери сознания', 'танцевальная дуэль', 'русские народные танцы', 'танцевальный биф', 'танцы животных'],
	'Наука': ['наука', 'научные факты', 'открытия', 'научпоп', 'исследования', 'альберт эйнштейн', 'никола тесла', 'законы ньютона', 'исследования про вред мефедрона', 'исследования про пользу алкоголя', 'нобелевские премии ученым'],
	'Политика': ['политика', 'политические новости', 'выборы', 'дебаты', 'байден', 'трамп', 'svo', 'za наших', 'путин', 'война', 'палестина израиль конфликт', 'герои россии', 'zov', 'война в индии', 'конфликт тайвань', 'наш донбасс', 'медведев о трампе', 'гойда', 'санкции против россии', 'конфликт армения азербайджан', 'война за нагорный карабах', 'колумбия готовится к войне','Владимир Путин 2025', 'Сергей Лавров 2025', 'Мария Захарова 2025','Алексей Венедиктов 2025', 'Михаил Ходорковский 2025','Алексей Навальный 2025', 'Евгений Пригожин','Дональд Трамп 2025', 'Джо Байден 2025', 'Камала Харрис 2025','Владимир Зеленский 2025', 'Эммануэль Макрон 2025','Риши Сунак 2025', 'Олаф Шольц 2025', 'Си Цзиньпин 2025','Никита Михалков Бесогон', 'Дмитрий Медведев интервью 2025','Сергей Миронов 2025', 'Григорий Явлинский 2025','Соловьев LIVE 2025', 'Вечер с Владимиром Соловьевым 2025','60 минут ток-шоу 2025', 'Время покажет 2025', 'Место встречи 2025','НТВ политика 2025', 'Первый канал новости 2025','РБК политика 2025', 'РБК интервью 2025', 'Коммерсантъ политика 2025','Meduza политика 2025', 'BBC News политика 2025', 'CNN News политика 2025','Bloomberg политика 2025', 'Euronews политика 2025', 'DW политика 2025','Reuters политика 2025', 'The Guardian политика 2025','санкции 2025', 'нефть и газ политика 2025', 'энергетическая политика 2025','экономическая политика 2025', 'инфляция 2025', 'кризис 2025','рынки и политика 2025', 'финансовые новости 2025', 'бюджет 2025','торговые отношения 2025', 'новости ОПЕК 2025', 'новости ВТО 2025',],
	'Тренировки': ['тренировка', 'фитнес тренировка', 'упражнения', 'спортзал', 'workout', 'спартак фитнесс тренер из симферополя', 'тренировка бицепса', 'тренировка спины', 'тренировка ног', 'тренировки для похудения', 'как научиться подтягиваться', 'тренировка макгрегора', 'как тренировался шварцнегер'],
	'Обзоры': ['обзор', 'review', 'обзор товара', 'тест', 'сравнение', 'обзор хаты', 'обзор площадки', 'обзор на машину', 'обзор на айфон', 'обзор на реплику apple watch', 'обзор air pods', 'обзор компьютерных мышек', 'обзор треков элджея', 'обзор на гроб', 'обзор на наушники', 'обзор на литенерджи'],
	'Рецепты': ['рецепт', 'рецепт блюда', 'кулинарный рецепт', 'как приготовить', 'рецепт пельменей', 'рецепт лазаньи', 'древнерусские рецепты', 'рецепт пирога', 'простые рецепты торта'],
	'Стримы': ['стрим', 'стрим игра', 'live stream', 'прямой эфир', 'трансляция', 'стрим по гта', 'стрим дота'],
	'Видеоблоги': ['влог', 'видеоблог', 'vlog', 'дневник', 'блогер'],
	'Образование': ['образование', 'обучение онлайн', 'урок онлайн', 'школа', 'открытый урок подготовки к егэ', 'подготовка к егэ', 'подготовка к огэ', 'подготовка к впр'],
	'Документальные': ['документальный фильм', 'докфильм', 'история', 'документалка ВОВ', 'документальный фильм 1941', 'документалка про футболиста'],
	'Анимация': ['анимация', 'мультфильм', 'мультик', 'аниме', 'cartoon', 'смешарики', 'пин код', 'мимимишки', 'маша и медведь', 'русские мультфильмы', 'три кота', 'лунтик', 'фиксики', 'барбоскины', 'три богатыря','анимация 2025', 'мультфильмы 2025', 'аниме 2025', 'cartoon 2025', 'мультсериалы 2025', 'Pixar 2025',' Disney 2025', ' DreamWorks 2025', 'новые мультфильмы 2025','аниме новинки 2025', 'лучшие мультфильмы 2025', 'мультфильмы для детей 2025','мультфильмы для взрослых 2025', '3D анимация 2025', '2D анимация 2025','анимация короткометражка','мультики онлайн 2025','Смешарики новые серии 2025', 'Пин код 2025', 'Мимимишки новые серии 2025','Маша и Медведь', 'Фиксики новые серии', 'Барбоскины новые серии','Лунтик новые серии 2025', 'Белка и Стрелка приключения 2025','Щенячий патруль' , 'Город героев мультсериал', 'Губка Боб новые серии','Миньоны мультфильм', 'Angry Birds мультфильм', 'Том и Джерри 2025','Покемон 2025', 'Ben 10 новые серии', 'Черепашки-ниндзя 2025','Мультфильмы про животных', 'мультики для малышей', 'детские песни мультфильмы','аниме мемы 2025', 'аниме лучшие моменты 2025','аниме ending 2025','One Piece 2025', 'Attack on Titan финал', 'Naruto', 'Boruto ','Jujutsu Kaisen ', 'Chainsaw Man ', 'Demon Slayer ','Tokyo Ghoul', 'My Hero Academia', 'Spy x Family ','Death Note 2025', 'Hunter x Hunter 2025', 'Black Clover 2025','Shrek', 'Shrek 5 трейлер', 'Kung Fu Panda 4', 'Minions 2025','Encanto 2 2025', 'Frozen 3 трейлер', 'Moana 2 трейлер','Inside Out 2 трейлер', 'Toy Story 5', 'Monsters at Work 2025','How to Train Your Dragon live action ', 'Spider-Man Across the Spider-Verse','Teenage Mutant Ninja Turtles 2025', 'Sonic 3 2025', 'Mario Movie 2 2025'],
	'Концерты': ['концерт', 'выступление', 'live концерт', 'музыкальный концерт', 'концерт инстасамка', 'концерт егор крид'],
	'Интервью': ['интервью', 'интервью ургант', 'интервью камеди', 'интервью у иностранных звезд', 'интервью со звездой', 'интервью вася вписка'],
	'Мемы': ['мем', 'мемы', 'мемный', 'приколы', 'арбуз арбуз привет', 'пять десять пятого', 'это камера? нет это сова', 'мем наполеон'],
	'Туториалы': ['туториал', 'обучающее видео', 'как сделать', 'инструкция', 'урок', 'тутор'],
	'Лайфхаки': ['лайфхак', 'совет', 'полезные советы', 'проверка лайфхаков', 'жизненный совет']
}

# Подсчитываем общее количество запросов (для информации)
total_queries = sum(len(queries) for queries in category_mapping.values())

In [9]:
def collect_videos_with_categories(category_mapping):
    all_videos = []

    for category, queries in category_mapping.items():
        for query in queries:
            offset = 0
            while True:
                result = get_videos_by_search(query, count=200, offset=offset)
                if not result or 'response' not in result:
                    break

                items = result['response'].get('items', [])
                if not items:
                    break

                for video in items:
                    parsed = parse_video_data(video)
                    parsed['search_query'] = query
                    parsed['category'] = category
                    all_videos.append(parsed)

                offset += len(items)
                total_count = result['response'].get('count', 0)
                if offset >= total_count or len(items) < 200:
                    break

                time.sleep(0.5)

            time.sleep(0.5)

        time.sleep(1)

    df = pd.DataFrame(all_videos)
    df = df.drop_duplicates(subset=['video_id'], keep='first')
    return df

In [10]:
# Запускаем расширенный сбор данных
vk_videos_df_extended = collect_videos_with_categories(category_mapping)


KeyboardInterrupt: 

In [ ]:
vk_videos_df_extended['category'].value_counts()

Размер датасета: (15157, 20)
Категорий: 30
Распределение по категориям:
category
Автомобили         227
Анимация           529
Видеоблоги          27
Документальные      87
Животные           268
Здоровье            75
Игры               789
Интервью            64
Кино               416
Комедия            592
Концерты            47
Кулинария          306
Лайфхаки            44
Мемы                46
Музыка            2676
Наука              121
Новости            765
Обзоры              63
Образование         50
Обучение           674
Политика          1029
Путешествия       1362
Развлечения       1429
Рецепты             35
Спорт             2759
Стримы              28
Танцы              172
Технологии         382
Тренировки          76
Туториалы           19
Name: count, dtype: int64


In [ ]:
vk_videos_df_extended['engagement_rate'] = (
    (vk_videos_df_extended['likes'] / vk_videos_df_extended['views'] * 100).round(2)
).fillna(0)

vk_videos_df_extended['duration_minutes'] = (vk_videos_df_extended['duration'] / 60).round(2)

vk_videos_df_extended['comments_per_1k_views'] = (
    (vk_videos_df_extended['comments'] / (vk_videos_df_extended['views'] / 1000)).round(2)
).fillna(0)

In [ ]:
vk_videos_df_extended.to_csv('vk_videos_dataset_extended.csv', index=False, encoding='utf-8-sig')
vk_videos_df_extended.to_excel('vk_videos_dataset_extended.xlsx', index=False, engine='openpyxl')